In [2]:
import pandas as pd

In [7]:
raw_data = pd.read_csv("./data/ARM_January 17, 2024_05.53.csv", skiprows=[2])

In [9]:
raw_data = raw_data[raw_data["Status"] == "IP Address"]

In [12]:
raw_data.drop(
    columns=[
        "Status",
        "IPAddress",
        "RecipientLastName",
        "RecipientFirstName",
        "RecipientEmail",
        "ExternalReference",
        "LocationLatitude",
        "LocationLongitude",
        "DistributionChannel",
        "UserLanguage",
    ]
)

,StartDate,EndDate,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,Q_RecaptchaScore,QC1,QD1,...,Q77,QS8,QS9,QS10,Q74,Q79,QS11,Q52,Q80,Q81
19,2024-01-16 05:19:14,2024-01-16 05:33:55,100,881,True,2024-01-16 05:33:56,R_72uxJTbT26KJZSJ,1,Yes,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,2024-01-16 05:57:58,2024-01-16 06:07:10,100,551,True,2024-01-16 06:07:11,R_1xlLKSLVgimtnPj,0.8999999761581421,Yes,Male,...,Satisfied,Q2,October,January-February,Somewhat confident,Neither dissatisfied or satisfied,NaN,NaN,NaN,NaN
21,2024-01-16 05:58:26,2024-01-16 06:08:48,100,622,True,2024-01-16 06:08:49,R_6nZIlRrs9aUGKHL,1,Yes,Male,...,Very satisfied,Q2,February,November-December,Neither confident or unsure,Dissatisfied,3rd of November,8th of November,Not confident at all,Very dissatisfied
22,2024-01-16 06:31:46,2024-01-16 06:44:03,100,736,True,2024-01-16 06:44:03,R_7PplWzAG0igeOml,1,Yes,Male,...,Very dissatisfied,Q2,February,November-December,Somewhat confident,Neither dissatisfied or satisfied,10th of November,24th of November,Somewhat confident,Neither dissatisfied or satisfied
23,2024-01-16 05:21:40,2024-01-16 08:17:14,100,10534,True,2024-01-16 08:17:15,R_2nipTYf8mhH2Yus,0.8999999761581421,Yes,Non-binary / third gender,...,Very satisfied,Q2,February,November-December,Somewhat confident,Satisfied,10th of November,24th of November,Somewhat confident,Satisfied
24,2024-01-16 09:26:32,2024-01-16 09:34:10,100,457,True,2024-01-16 09:34:11,R_1dy2jAJUx1DgK6R,0.800000011920929,Yes,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2024-01-16 09:27:48,2024-01-16 09:36:15,100,506,True,2024-01-16 09:36:15,R_2esfOQZiXvHpgod,0.8999999761581421,Yes,Male,...,Very dissatisfied,Q1,October,January-February,Somewhat confident,Satisfied,10th of November,29th of November,Neither confident or unsure,Dissatisfied
26,2024-01-17 03:09:03,2024-01-17 03:19:06,100,602,True,2024-01-17 03:19:06,R_2z5zMoIKEHtkx0S,0.800000011920929,Yes,Female,...,Very satisfied,Q2,February,January-February,Not confident,Neither dissatisfied or satisfied,10th of November,24th of November,Confident,Very satisfied
27,2024-01-17 03:25:49,2024-01-17 03:28:42,100,173,True,2024-01-17 03:28:43,R_6QE4FQzYCk4GaNV,1,Yes,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,2024-01-17 03:25:40,2024-01-17 03:33:06,100,446,True,2024-01-17 03:33:06,R_5IGX6rHpxzyVRGc,0.8999999761581421,Yes,Female,...,Dissatisfied,Q3,October,November-December,Not confident at all,Very dissatisfied,10th of November,24th of November,Not confident at all,Dissatisfied


In [26]:
demographics = raw_data.loc[:, :"Q_RecaptchaScore"]

complex_responses = raw_data.loc[:, "QC1.1":"Q69"]
simple_responses = raw_data.loc[:, "QS1":]

complex_responses.dropna(thresh=10)
simple_responses.dropna(thresh=10)

,QS1,QS2,QS3,Q73,Q72,QS4_1,QS4_2,QS4_3,QS4_4,QS4_5,...,Q77,QS8,QS9,QS10,Q74,Q79,QS11,Q52,Q80,Q81
20,False,False,True,Neither confident or unsure,Neither dissatisfied or satisfied,1,5,4,3,2,...,Satisfied,Q2,October,January-February,Somewhat confident,Neither dissatisfied or satisfied,NaN,NaN,NaN,NaN
21,True,False,False,Somewhat confident,Dissatisfied,1,5,4,3,2,...,Very satisfied,Q2,February,November-December,Neither confident or unsure,Dissatisfied,3rd of November,8th of November,Not confident at all,Very dissatisfied
22,False,False,True,Somewhat confident,Dissatisfied,1,4,5,3,2,...,Very dissatisfied,Q2,February,November-December,Somewhat confident,Neither dissatisfied or satisfied,10th of November,24th of November,Somewhat confident,Neither dissatisfied or satisfied
23,False,False,False,Somewhat confident,Neither dissatisfied or satisfied,NaN,NaN,NaN,NaN,NaN,...,Very satisfied,Q2,February,November-December,Somewhat confident,Satisfied,10th of November,24th of November,Somewhat confident,Satisfied
25,False,False,True,Somewhat confident,Satisfied,1,5,4,3,2,...,Very dissatisfied,Q1,October,January-February,Somewhat confident,Satisfied,10th of November,29th of November,Neither confident or unsure,Dissatisfied
26,False,False,False,Not confident at all,Dissatisfied,1,5,4,3,2,...,Very satisfied,Q2,February,January-February,Not confident,Neither dissatisfied or satisfied,10th of November,24th of November,Confident,Very satisfied
28,True,False,False,Not confident,Dissatisfied,1,5,4,3,2,...,Dissatisfied,Q3,October,November-December,Not confident at all,Very dissatisfied,10th of November,24th of November,Not confident at all,Dissatisfied
29,False,False,True,Somewhat confident,Satisfied,1,5,4,3,2,...,Satisfied,Q1,October,January-February,Somewhat confident,Dissatisfied,10th of November,29th of November,Neither confident or unsure,Dissatisfied
32,False,False,True,Somewhat confident,Neither dissatisfied or satisfied,1,5,4,3,2,...,Neither dissatisfied or satisfied,Q2,October,January-February,Somewhat confident,Satisfied,10th of November,24th of November,Somewhat confident,Satisfied


In [30]:
answer_sheet = pd.read_excel("./data/answer_sheet.xlsx")

In [31]:
answer_sheet.head()

,Q1.1,Q1.2,Q1.3,Q2.1,Q2.2,Q2.3,Q2.4,Q2.5,Q3.1,Q3.2,Q3.3,Q3.4,Q3.4.1,Q3.5,Q4.1,Q4.2,Q4.3,Q5.1,Q5.2
0,False,False,True,Executive,2,1,3,4,1,5,4,3,2,34,Q2,Febuary,January-Febuary,10th of November,24th of November


In [ ]:
likert_a = pd.DataFrame({})